<a href="https://colab.research.google.com/github/shruti-singh/PaperAcceptancePrediction/blob/master/zeel/refextract/Refextract_fromFilenames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y python-magic
!pip install refextract
!pip install tika

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-mgc libmagic1 python-magic
0 upgraded, 3 newly installed, 0 to remove and 25 not upgraded.
Need to get 262 kB of archives.
After this operation, 5,255 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.3 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.3 [68.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-magic all 2:0.4.15-1 [9,192 B]
Fetched 262 kB in 2s (109 kB/s)
Selecting previously unselected package libmagic-mgc.
(Reading 

In [0]:
#import refextract
#from tika import parser
from glob import glob
import numpy as np
from subprocess import Popen, PIPE
import pandas as pd
import sys
import os
import pickle

In [0]:
year = '2020'

In [0]:
fnames = glob('/content/drive/My Drive/Colab Notebooks/ML Project/data/papers/'+year+'/*.pdf')
length = len(fnames)
print(length)

1374


In [0]:
N = 50 # Number of workers to run parallelly
chunks = np.linspace(0, length, N).astype(int)

In [0]:
chunks[:10]

array([  0,  28,  56,  84, 112, 140, 168, 196, 224, 252])

In [0]:
path = '/content/drive/My Drive/SEM-1 ML Project'

In [0]:
# to start all workers
procs = []
p_file = path+'tmp/runner.py'
for i,j in zip(chunks,chunks[1:]):
  if os.path.exists(path+'tmp/'+str(i)+'-'+str(j)+'.pickle'):
    continue
  procs.append(Popen(['python3',p_file,str(i),str(j),year], stdout=PIPE, stderr=PIPE))
  sys.stdout.write('\r'+str(i)+'_'+str(j)+' started\n')
  sys.stdout.flush()

28_56 started
140_168 started
252_280 started
280_308 started
308_336 started
336_364 started
392_420 started
448_476 started
504_532 started
560_588 started
616_644 started
672_701 started
701_729 started
925_953 started
1037_1065 started
1093_1121 started
1177_1205 started
1261_1289 started
1289_1317 started


In [0]:
# Mass status Check of workers
# Run this multiple times to check the status of workers
status = []
c = 0
for proc in procs:
  try:
    state = proc.poll()
    if state == 1:
      print(proc.communicate()[1])
    status.append(state)
  except:
    c += 1
print(c)
pd.Series(status).value_counts()

0


-9    19
dtype: int64

In [0]:
#################################################
##                                             ##
################### WARNING #####################
##                                             ##
#################################################
# Killing all the workers in case of problems
for proc in procs:
  proc.kill()

In [0]:
ref_obs = glob(path+'/tmp/*.pickle')

In [15]:
#########
# To be corrected---
#########
def validate(ref):
  if 'year' in ref.keys():
    if 'Published as a conference paper at ICLR' not in ref['raw_ref']:
      if int(ref['year'][0]) in range(1800,2021):
        return True
  return False

i = 0
library = {}
for obj in ref_obs:
  try:
    ref_dict = pd.read_pickle(obj)
  except EOFError:
    os.remove(obj)
    continue
  for refs in ref_dict:
    library.update({refs.replace('.pdf',''): [ref['year'][0] for ref in refs if validate(ref)]})
  print(i)
  i += 1
len(library)

AttributeError: ignored

In [0]:
ref_df = pd.DataFrame()
ref_df['forum'] = library.keys()
ref_df['ref_years'] = [[int(i) for i in j] for j in library.values()]
ref_df.set_index('forum', inplace=True)
print(ref_df.head(2))
print('\n\n')

# Preprocessing
ref_df['ref_ylen'] = ref_df['ref_years'].apply(len) # Number of refs
ref_df['ref_latest'] = ref_df['ref_years'].apply(lambda x: max(x+[1992])) #latest ref
print(ref_df['ref_latest'].value_counts())
print('\n\n')
print(ref_df.head(2))

                                                    ref_years
forum                                                        
SkVhlh09tX  [2017, 2016, 2015, 2018, 2013, 2018, 2015, 201...
Hyl7ygStwB  [2015, 2019, 2019, 2014, 2014, 2019, 2015, 201...



2019    142
2018     61
2020      9
2017      6
2015      1
2009      1
Name: ref_latest, dtype: int64



                                                    ref_years  ...  ref_latest
forum                                                          ...            
SkVhlh09tX  [2017, 2016, 2015, 2018, 2013, 2018, 2015, 201...  ...        2019
Hyl7ygStwB  [2015, 2019, 2019, 2014, 2014, 2019, 2015, 201...  ...        2019

[2 rows x 3 columns]


In [0]:
# Assigning these features to original csv
data['ref_latest'] = ref_df['ref_latest']
data['ref_len'] = ref_df['ref_ylen']
data['ref_years'] = ref_df['ref_years']

In [0]:
# Saving the updated csv
data_csv.to_pickle('/content/drive/My Drive/Colab Notebooks/ML Project/data/data_csv_refextracted-2017-20.pkl')